### RULE 

第 3 列更大时，第 9 列也更大

### 训练信息

In [ ]:
% 训练信息
rule_model_train_time      = "3h25min"
log_path                   =  "/Users/luminshen/Desktop/模型/rule model/3大，9大/events.out.tfevents.1619669644.luminshendeMacBook-Pro.local"          
model_path                 = "/Users/luminshen/Desktop/模型/rule model/3大，9大/Adult_500_7_7"
train_data_path            = "/Users/luminshen/Desktop/模型/rule model/3大，9大/traindata"

gan_train_time             = "7h"
gan_path                   = "/Users/luminshen/Desktop/模型/new gan/3>, 9>"

### 全局参数

In [1]:
COL_1 = 3
COL_2 = 9

### GT 中的 error rate

In [2]:
import numpy as np
import pandas as pd

GROUND_TRUTH_PATH = "/Users/luminshen/Documents/代码/PycharmProjects/Research/-GAN-/Table-GAN/tableGAN/data/Adult/Adult.csv"

def verify_ground_truth_rule():
    file = list(np.array(pd.read_csv(GROUND_TRUTH_PATH, sep=',')))
    col_1 = COL_1
    col_2 = COL_2
    count = 0
    error_list = []
    for i, _ in enumerate(file):
        isOk = True
        t = 0
        for j, _ in enumerate(file):
            if file[i][col_1] > file[j][col_1] and file[i][col_2] > file[j][col_2]:
                error_list.append((i+2, j+2))
                t -= 1
                if t < 0:
                    isOk = False
                    count += 1
                    break
    print("error: {}, error rate:{}".format(count, count / len(file)))
    return count

error_GT = verify_ground_truth_rule()

error: 1123, error rate:0.5615


### 给数据添加标签

In [3]:
X_OUTPUT = "/Users/luminshen/Documents/代码/PycharmProjects/Research/-GAN-/Table-GAN/tableGAN/data/Adult/Adult_rulemodel.csv"
Y_OUTPUT = "/Users/luminshen/Documents/代码/PycharmProjects/Research/-GAN-/Table-GAN/tableGAN/data/Adult/Adult_rulemodel_labels.csv"

def generate_data():
    file = list(np.array(pd.read_csv(GROUND_TRUTH_PATH, sep=',')))
    positive = []
    labels = []
    negative = []
    col_1 = COL_1
    col_2 = COL_2
    error = 0
    for i, _ in enumerate(file):
        isOK = True
        for j, _ in enumerate(file):
            if file[i][col_1] > file[j][col_1] and file[i][col_2] > file[j][col_2]:
                negative.append(file[i])
                labels.append([1])
                error += 1
                isOK = False
                break
        if isOK:
            positive.append(file[i])
            labels.append([0])

    X = np.array(positive + negative)
    y = np.array(labels)
    X_out = pd.DataFrame(X)
    y_out = pd.DataFrame(y)
    X_out.to_csv(X_OUTPUT, index=False, sep=',')
    y_out.to_csv(Y_OUTPUT, index=False, sep=',')
    print("输出完成！Xshape:{}, yshape:{} error: {}, error rate:{}".format(X.shape, y.shape, error, error / len(X)))
    return error

error_GD = generate_data()

输出完成！Xshape:(2000, 14), yshape:(2000, 1) error: 1123, error rate:0.5615


In [4]:
assert(error_GT == error_GD)

### 验证 Fake Data

In [58]:
# >, >
OUTPUT_TMP_PATH = "/Users/luminshen/Documents/代码/PycharmProjects/Research/-GAN-/Table-GAN/tableGAN/samples/Adult/Adult_OI_11_00_fake.csv"
OUTPUT_PATH = ""
ORIGINAL_PATH = "/Users/luminshen/Desktop/模型/original gan/600分钟/Adult_OI_11_00_fake.csv"

def verify_rule(path, ratio, c1, c2, name):
    if ratio == -1:
        file = list(np.array(pd.read_csv(path, sep=',')))
    else:
        file = list(np.array(pd.read_csv(path, sep=';')))
    col_1 = c1
    col_2 = c2
    count = 0
    error_list = []
    for i, _ in enumerate(file):
        isOk = True
        t = 0
        for j, _ in enumerate(file):
            if file[i][col_1] > file[j][col_1] and file[i][col_2] > file[j][col_2]:
                t -= 1
                if t < 0:
                    count += 1

    error_rate = count / len(file)**2
    if ratio == -1:
        print("ground truth: error: {:8.0f}, error rate:{:8.6f}".format(count, count / len(file)**2))
        return count / len(file)**2    
    
    else:    
        if error_rate - ratio > 0:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:+{:8.6f}".format(name, count, error_rate, error_rate - ratio))
        else:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:{:8.6f}".format(name, count, error_rate, error_rate - ratio))

        return error_rate - ratio


ratio = verify_rule(GROUND_TRUTH_PATH, -1, 0, 9, "")
verify_rule(OUTPUT_TMP_PATH, ratio, 0, 9, "fake")
verify_rule(ORIGINAL_PATH, ratio, 0, 9, "original")
print("------------------------------------------------")

count_fake = 0
count_original = 0
sum_fake = 0
sum_original = 0

for c1 in range(10):
    for c2 in range(10):
        if c1 == c2:
            continue
        ratio = verify_rule(GROUND_TRUTH_PATH, -1, c1, c2 , "")
        diff_fake = verify_rule(OUTPUT_TMP_PATH, ratio, c1, c2, "fake")
        diff_original = verify_rule(ORIGINAL_PATH, ratio, c1, c2, "original")
        sum_fake += abs(diff_fake)
        sum_original += abs(diff_original)
        if abs(diff_fake) > abs(diff_original):
            count_original += 1
        else:
            count_fake += 1
        print("------------------------------------------------ {}:{}".format(count_fake, count_original))

print("fake: {}, original: {}, ratio: {}, fake_diff: {}. original_diff: {}".format(count_fake, count_original, count_fake/count_original, sum_fake, sum_original))



ground truth: error:   465612, error rate:0.116403
fake        : error:   480281, error rate:0.120070, diff:+0.003667
original    : error:   762362, error rate:0.190590, diff:+0.074187
------------------------------------------------
ground truth: error:   581265, error rate:0.145316
fake        : error:   787299, error rate:0.196825, diff:+0.051508
original    : error:   683094, error rate:0.170773, diff:+0.025457
------------------------------------------------ 0:1
ground truth: error:   920766, error rate:0.230191
fake        : error:   934162, error rate:0.233541, diff:+0.003349
original    : error:   627810, error rate:0.156952, diff:-0.073239
------------------------------------------------ 1:1
ground truth: error:   895965, error rate:0.223991
fake        : error:   896694, error rate:0.224173, diff:+0.000182
original    : error:  1119328, error rate:0.279832, diff:+0.055841
------------------------------------------------ 2:1
ground truth: error:   796256, error rate:0.199064
f

original    : error:   825719, error rate:0.206430, diff:+0.054325
------------------------------------------------ 26:8
ground truth: error:   210192, error rate:0.052548
fake        : error:   167796, error rate:0.041949, diff:-0.010599
original    : error:    72193, error rate:0.018048, diff:-0.034500
------------------------------------------------ 27:8
ground truth: error:   333956, error rate:0.083489
fake        : error:   328335, error rate:0.082084, diff:-0.001405
original    : error:   672932, error rate:0.168233, diff:+0.084744
------------------------------------------------ 28:8
ground truth: error:   796256, error rate:0.199064
fake        : error:   911107, error rate:0.227777, diff:+0.028713
original    : error:   436874, error rate:0.109218, diff:-0.089845
------------------------------------------------ 29:8
ground truth: error:   476761, error rate:0.119190
fake        : error:   653751, error rate:0.163438, diff:+0.044247
original    : error:   360010, error rate:0.

ground truth: error:   842151, error rate:0.210538
fake        : error:   770216, error rate:0.192554, diff:-0.017984
original    : error:   266794, error rate:0.066698, diff:-0.143839
------------------------------------------------ 55:14
ground truth: error:   667756, error rate:0.166939
fake        : error:   769460, error rate:0.192365, diff:+0.025426
original    : error:   226192, error rate:0.056548, diff:-0.110391
------------------------------------------------ 56:14
ground truth: error:   248943, error rate:0.062236
fake        : error:   263198, error rate:0.065799, diff:+0.003564
original    : error:    13872, error rate:0.003468, diff:-0.058768
------------------------------------------------ 57:14
ground truth: error:   304312, error rate:0.076078
fake        : error:   331046, error rate:0.082762, diff:+0.006683
original    : error:   488103, error rate:0.122026, diff:+0.045948
------------------------------------------------ 58:14
ground truth: error:   254600, error rat

In [57]:
# =, !=
OUTPUT_TMP_PATH = "/Users/luminshen/Documents/代码/PycharmProjects/Research/-GAN-/Table-GAN/tableGAN/samples/Adult/Adult_OI_11_00_fake.csv"
OUTPUT_PATH = ""
ORIGINAL_PATH = "/Users/luminshen/Desktop/模型/original gan/600分钟/Adult_OI_11_00_fake.csv"

def verify_rule(path, ratio, c1, c2, name):
    if ratio == -1:
        file = list(np.array(pd.read_csv(path, sep=',')))
    else:
        file = list(np.array(pd.read_csv(path, sep=';')))
    col_1 = c1
    col_2 = c2
    count = 0
    error_list = []
    for i, _ in enumerate(file):
        isOk = True
        t = 0
        for j, _ in enumerate(file):
            if file[i][col_1] == file[j][col_1] and file[i][col_2] != file[j][col_2]:
                t -= 1
                if t < 0:
                    count += 1

    error_rate = count / len(file)**2
    if ratio == -1:
        print("ground truth: error: {:8.0f}, error rate:{:8.6f}".format(count, count / len(file)**2))
        return count / len(file)**2    
    
    else:    
        if error_rate - ratio > 0:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:+{:8.6f}".format(name, count, error_rate, error_rate - ratio))
        else:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:{:8.6f}".format(name, count, error_rate, error_rate - ratio))

        return error_rate - ratio


ratio = verify_rule(GROUND_TRUTH_PATH, -1, 0, 9, "")
verify_rule(OUTPUT_TMP_PATH, ratio, 0, 9, "fake")
verify_rule(ORIGINAL_PATH, ratio, 0, 9, "original")
print("------------------------------------------------")

count_fake = 0
count_original = 0
sum_fake = 0
sum_original = 0

for c1 in range(10):
    for c2 in range(10):
        if c1 == c2:
            continue
        ratio = verify_rule(GROUND_TRUTH_PATH, -1, c1, c2 , "")
        diff_fake = verify_rule(OUTPUT_TMP_PATH, ratio, c1, c2, "fake")
        diff_original = verify_rule(ORIGINAL_PATH, ratio, c1, c2, "original")
        sum_fake += abs(diff_fake)
        sum_original += abs(diff_original)
        if abs(diff_fake) > abs(diff_original):
            count_original += 1
        else:
            count_fake += 1
        print("------------------------------------------------ {}:{}".format(count_fake, count_original))

print("fake: {}, original: {}, ratio: {}, fake_diff: {}. original_diff: {}".format(count_fake, count_original, count_fake/count_original, sum_fake, sum_original))


ground truth: error:    35988, error rate:0.008997
fake        : error:    32246, error rate:0.008061, diff:-0.000936
original    : error:    54582, error rate:0.013645, diff:+0.004648
------------------------------------------------
ground truth: error:    39658, error rate:0.009914
fake        : error:    60164, error rate:0.015041, diff:+0.005127
original    : error:    85712, error rate:0.021428, diff:+0.011513
------------------------------------------------ 1:0
ground truth: error:    85084, error rate:0.021271
fake        : error:    78548, error rate:0.019637, diff:-0.001634
original    : error:   316900, error rate:0.079225, diff:+0.057954
------------------------------------------------ 2:0
ground truth: error:    66284, error rate:0.016571
fake        : error:    65228, error rate:0.016307, diff:-0.000264
original    : error:   143984, error rate:0.035996, diff:+0.019425
------------------------------------------------ 3:0
ground truth: error:    66284, error rate:0.016571
f

original    : error:   602832, error rate:0.150708, diff:+0.008726
------------------------------------------------ 17:17
ground truth: error:   214504, error rate:0.053626
fake        : error:   166588, error rate:0.041647, diff:-0.011979
original    : error:   187914, error rate:0.046978, diff:-0.006648
------------------------------------------------ 17:18
ground truth: error:   340830, error rate:0.085208
fake        : error:   259872, error rate:0.064968, diff:-0.020240
original    : error:   238462, error rate:0.059616, diff:-0.025592
------------------------------------------------ 18:18
ground truth: error:   765356, error rate:0.191339
fake        : error:   586506, error rate:0.146626, diff:-0.044713
original    : error:   696798, error rate:0.174200, diff:-0.017140
------------------------------------------------ 18:19
ground truth: error:   383530, error rate:0.095882
fake        : error:   459668, error rate:0.114917, diff:+0.019035
original    : error:   350466, error rat

ground truth: error:   216244, error rate:0.054061
fake        : error:   506150, error rate:0.126537, diff:+0.072476
original    : error:   377112, error rate:0.094278, diff:+0.040217
------------------------------------------------ 42:27
ground truth: error:   972256, error rate:0.243064
fake        : error:   881548, error rate:0.220387, diff:-0.022677
original    : error:   660502, error rate:0.165126, diff:-0.077938
------------------------------------------------ 43:27
ground truth: error:   249692, error rate:0.062423
fake        : error:   146186, error rate:0.036547, diff:-0.025876
original    : error:   225856, error rate:0.056464, diff:-0.005959
------------------------------------------------ 43:28
ground truth: error:   205416, error rate:0.051354
fake        : error:   277808, error rate:0.069452, diff:+0.018098
original    : error:   257188, error rate:0.064297, diff:+0.012943
------------------------------------------------ 43:29
ground truth: error:  2862330, error rat

In [59]:
# >, !=
OUTPUT_TMP_PATH = "/Users/luminshen/Documents/代码/PycharmProjects/Research/-GAN-/Table-GAN/tableGAN/samples/Adult/Adult_OI_11_00_fake.csv"
OUTPUT_PATH = ""
ORIGINAL_PATH = "/Users/luminshen/Desktop/模型/original gan/600分钟/Adult_OI_11_00_fake.csv"

def verify_rule(path, ratio, c1, c2, name):
    if ratio == -1:
        file = list(np.array(pd.read_csv(path, sep=',')))
    else:
        file = list(np.array(pd.read_csv(path, sep=';')))
    col_1 = c1
    col_2 = c2
    count = 0
    error_list = []
    for i, _ in enumerate(file):
        isOk = True
        t = 0
        for j, _ in enumerate(file):
            if file[i][col_1] > file[j][col_1] and file[i][col_2] != file[j][col_2]:
                t -= 1
                if t < 0:
                    count += 1

    error_rate = count / len(file)**2
    if ratio == -1:
        print("ground truth: error: {:8.0f}, error rate:{:8.6f}".format(count, count / len(file)**2))
        return count / len(file)**2    
    
    else:    
        if error_rate - ratio > 0:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:+{:8.6f}".format(name, count, error_rate, error_rate - ratio))
        else:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:{:8.6f}".format(name, count, error_rate, error_rate - ratio))

        return error_rate - ratio


ratio = verify_rule(GROUND_TRUTH_PATH, -1, 0, 9, "")
verify_rule(OUTPUT_TMP_PATH, ratio, 0, 9, "fake")
verify_rule(ORIGINAL_PATH, ratio, 0, 9, "original")
print("------------------------------------------------")

count_fake = 0
count_original = 0
sum_fake = 0
sum_original = 0

for c1 in range(10):
    for c2 in range(10):
        if c1 == c2:
            continue
        ratio = verify_rule(GROUND_TRUTH_PATH, -1, c1, c2 , "")
        diff_fake = verify_rule(OUTPUT_TMP_PATH, ratio, c1, c2, "fake")
        diff_original = verify_rule(ORIGINAL_PATH, ratio, c1, c2, "original")
        sum_fake += abs(diff_fake)
        sum_original += abs(diff_original)
        if abs(diff_fake) > abs(diff_original):
            count_original += 1
        else:
            count_fake += 1
        print("------------------------------------------------ {}:{}".format(count_fake, count_original))

print("fake: {}, original: {}, ratio: {}, fake_diff: {}. original_diff: {}".format(count_fake, count_original, count_fake/count_original, sum_fake, sum_original))



ground truth: error:   843622, error rate:0.210905
fake        : error:   857852, error rate:0.214463, diff:+0.003557
original    : error:   822165, error rate:0.205541, diff:-0.005364
------------------------------------------------
ground truth: error:   990644, error rate:0.247661
fake        : error:  1520998, error rate:0.380250, diff:+0.132589
original    : error:   973631, error rate:0.243408, diff:-0.004253
------------------------------------------------ 0:1
ground truth: error:  1956399, error rate:0.489100
fake        : error:  1957716, error rate:0.489429, diff:+0.000329
original    : error:  1837280, error rate:0.459320, diff:-0.029780
------------------------------------------------ 1:1
ground truth: error:  1573768, error rate:0.393442
fake        : error:  1658312, error rate:0.414578, diff:+0.021136
original    : error:  1365180, error rate:0.341295, diff:-0.052147
------------------------------------------------ 2:1
ground truth: error:  1573768, error rate:0.393442
f

original    : error:  1100680, error rate:0.275170, diff:-0.017256
------------------------------------------------ 17:17
ground truth: error:   429906, error rate:0.107477
fake        : error:   407739, error rate:0.101935, diff:-0.005542
original    : error:   410758, error rate:0.102690, diff:-0.004787
------------------------------------------------ 17:18
ground truth: error:   691201, error rate:0.172800
fake        : error:   744039, error rate:0.186010, diff:+0.013210
original    : error:   730225, error rate:0.182556, diff:+0.009756
------------------------------------------------ 17:19
ground truth: error:  1573768, error rate:0.393442
fake        : error:  1666431, error rate:0.416608, diff:+0.023166
original    : error:  1491593, error rate:0.372898, diff:-0.020544
------------------------------------------------ 17:20
ground truth: error:   818708, error rate:0.204677
fake        : error:  1321246, error rate:0.330311, diff:+0.125634
original    : error:   841254, error rat

ground truth: error:  1202174, error rate:0.300544
fake        : error:  1135318, error rate:0.283830, diff:-0.016714
original    : error:   851171, error rate:0.212793, diff:-0.087751
------------------------------------------------ 38:31
ground truth: error:  1320591, error rate:0.330148
fake        : error:  1388674, error rate:0.347168, diff:+0.017021
original    : error:   954690, error rate:0.238673, diff:-0.091475
------------------------------------------------ 39:31
ground truth: error:   412312, error rate:0.103078
fake        : error:   417940, error rate:0.104485, diff:+0.001407
original    : error:   391787, error rate:0.097947, diff:-0.005131
------------------------------------------------ 40:31
ground truth: error:   758908, error rate:0.189727
fake        : error:   735071, error rate:0.183768, diff:-0.005959
original    : error:   720862, error rate:0.180216, diff:-0.009512
------------------------------------------------ 41:31
ground truth: error:   525281, error rat

In [ ]:
# >, !=
OUTPUT_TMP_PATH = "/Users/luminshen/Documents/代码/PycharmProjects/Research/-GAN-/Table-GAN/tableGAN/samples/Adult/Adult_OI_11_00_fake.csv"
OUTPUT_PATH = ""
ORIGINAL_PATH = "/Users/luminshen/Desktop/模型/original gan/600分钟/Adult_OI_11_00_fake.csv"

def verify_rule(path, ratio, c1, c2, name):
    if ratio == -1:
        file = list(np.array(pd.read_csv(path, sep=',')))
    else:
        file = list(np.array(pd.read_csv(path, sep=';')))
    col_1 = c1
    col_2 = c2
    count = 0
    error_list = []
    for i, _ in enumerate(file):
        isOk = True
        t = 0
        for j, _ in enumerate(file):
            if file[i][col_1] > file[j][col_1] and file[i][col_2] != file[j][col_2]:
                t -= 1
                if t < 0:
                    count += 1

    error_rate = count / len(file)**2
    if ratio == -1:
        print("ground truth: error: {:8.0f}, error rate:{:8.6f}".format(count, count / len(file)**2))
        return count / len(file)**2    
    
    else:    
        if error_rate - ratio > 0:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:+{:8.6f}".format(name, count, error_rate, error_rate - ratio))
        else:
            print("{:12}: error: {:8.0f}, error rate:{:8.6f}, diff:{:8.6f}".format(name, count, error_rate, error_rate - ratio))

        return error_rate - ratio


ratio = verify_rule(GROUND_TRUTH_PATH, -1, 0, 9, "")
verify_rule(OUTPUT_TMP_PATH, ratio, 0, 9, "fake")
verify_rule(ORIGINAL_PATH, ratio, 0, 9, "original")
print("------------------------------------------------")

count_fake = 0
count_original = 0
sum_fake = 0
sum_original = 0

for c1 in range(10):
    for c2 in range(10):
        if c1 == c2:
            continue
        ratio = verify_rule(GROUND_TRUTH_PATH, -1, c1, c2 , "")
        diff_fake = verify_rule(OUTPUT_TMP_PATH, ratio, c1, c2, "fake")
        diff_original = verify_rule(ORIGINAL_PATH, ratio, c1, c2, "original")
        sum_fake += abs(diff_fake)
        sum_original += abs(diff_original)
        if abs(diff_fake) > abs(diff_original):
            count_original += 1
        else:
            count_fake += 1
        print("------------------------------------------------ {}:{}".format(count_fake, count_original))

print("fake: {}, original: {}, ratio: {}, fake_diff: {}. original_diff: {}".format(count_fake, count_original, count_fake/count_original, sum_fake, sum_original))


